In [1]:
import warnings
warnings.filterwarnings('ignore')

import os, gc, random
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import  log_loss
from sklearn.model_selection import StratifiedKFold

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from matplotlib import pyplot as plt
import seaborn as sns

from IPython.display import display
from sklearn.model_selection import train_test_split

# pd.options.display.max_rows = 10000
pd.options.display.max_columns = 1000
# pd.options.display.max_colwidth = 1000

In [2]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
SEED = 2021
# seed_everything(SEED)
TARGET = 'credit'

In [3]:
train = pd.read_pickle("data/train_adjusted.pkl")
test = pd.read_pickle("data/test_adjusted.pkl")
sub = pd.read_csv("data/sample_submission.csv")

# FE

In [4]:
df = pd.concat([train.drop(['index','credit'],1), test.drop(['index'],1)], axis=0)
df['DAYS_BIRTH_DAYS_EMPLOYED_ratio'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
df['income_total_DAYS_BIRTH_ratio'] = df['income_total'] / df['DAYS_BIRTH']
df['income_total_DAYS_EMPLOYED_ratio'] = df['income_total'] / df['DAYS_EMPLOYED']
df['parents'] = df['family_size'] - df['child_num']
df['income_total_family_size_ratio'] = df['income_total'] / df['family_size']

In [5]:
df.drop('card_ID',axis=1, inplace=True)

In [6]:
cat_cols = [x for x in df.columns if df[x].dtype == 'object']
num_cols = [x for x in df.columns if x not in cat_cols + [TARGET]]
feature_cols = num_cols + cat_cols
print(len(feature_cols), len(cat_cols), len(num_cols))

22 8 14


In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
lbe = LabelEncoder()
for col in cat_cols:
    df[col] = lbe.fit_transform(df[col])

In [9]:
from tensorflow import keras

encoding_dim = 64

def get_model(encoding_dim, dropout=.2):
    num_dim = len(num_cols)
    num_input = keras.layers.Input((num_dim,), name='num_input')
    cat_inputs = []
    cat_embs = []
    emb_dims = 0
    for col in cat_cols:
        cat_input = keras.layers.Input((1,), name=f'{col}_input')
        emb_dim = max(8, int(np.log2(1 + df[col].nunique()) * 4))
        cat_emb = keras.layers.Embedding(input_dim=df[col].max() + 1, output_dim=emb_dim)(cat_input)
        cat_emb = keras.layers.Dropout(dropout)(cat_emb)
        cat_emb = keras.layers.Reshape((emb_dim,))(cat_emb)

        cat_inputs.append(cat_input)
        cat_embs.append(cat_emb)
        emb_dims += emb_dim

    merged_inputs = keras.layers.Concatenate()([num_input] + cat_embs)

    encoded = keras.layers.Dense(encoding_dim * 3, activation='relu')(merged_inputs)
    encoded = keras.layers.Dropout(dropout)(encoded)
    encoded = keras.layers.Dense(encoding_dim * 2, activation='relu')(encoded)
    encoded = keras.layers.Dropout(dropout)(encoded)    
    encoded = keras.layers.Dense(encoding_dim, activation='relu')(encoded)
    
    decoded = keras.layers.Dense(encoding_dim * 2, activation='relu')(encoded)
    decoded = keras.layers.Dropout(dropout)(decoded)
    decoded = keras.layers.Dense(encoding_dim * 3, activation='relu')(decoded)
    decoded = keras.layers.Dropout(dropout)(decoded)    
    decoded = keras.layers.Dense(num_dim + emb_dims, activation='linear')(encoded)

    encoder = keras.Model([num_input] + cat_inputs, encoded)
    ae = keras.Model([num_input] + cat_inputs, decoded)
    ae.add_loss(keras.losses.mean_squared_error(merged_inputs, decoded))
    ae.compile(optimizer='adam')
    return ae, encoder

In [10]:
ae, encoder = get_model(encoding_dim)
ae.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
gender_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
car_input (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
reality_input (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
income_type_input (InputLayer)  [(None, 1)]          0                                            
____________________________________________________________________________________________

In [11]:
inputs = [df[num_cols].values] + [df[x].values for x in cat_cols]

In [12]:
inputs = [df[num_cols].values] + [df[x].values for x in cat_cols]
ae.fit(inputs, inputs,
      epochs=100,
      batch_size=16384,
      shuffle=True,
      validation_split=.2)

Epoch 1/100
2/2 [==============================] - 4s 2s/step - loss: 722106624.0000 - val_loss: 612847936.0000
Epoch 2/100
2/2 [==============================] - 0s 167ms/step - loss: 636383040.0000 - val_loss: 554735872.0000
Epoch 3/100
2/2 [==============================] - 0s 165ms/step - loss: 576393898.6667 - val_loss: 504098368.0000
Epoch 4/100
2/2 [==============================] - 0s 158ms/step - loss: 525833749.3333 - val_loss: 456188704.0000
Epoch 5/100
2/2 [==============================] - 0s 161ms/step - loss: 483799626.6667 - val_loss: 411222464.0000
Epoch 6/100
2/2 [==============================] - 0s 154ms/step - loss: 441225696.0000 - val_loss: 364452800.0000
Epoch 7/100
2/2 [==============================] - 0s 171ms/step - loss: 393959722.6667 - val_loss: 310705696.0000
Epoch 8/100
2/2 [==============================] - 0s 163ms/step - loss: 348037461.3333 - val_loss: 258736176.0000
Epoch 9/100
2/2 [==============================] - 0s 173ms/step - loss: 302641856.

Epoch 73/100
2/2 [==============================] - 0s 219ms/step - loss: 16172073.0000 - val_loss: 2642803.2500
Epoch 74/100
2/2 [==============================] - 0s 214ms/step - loss: 15738359.3333 - val_loss: 2574195.5000
Epoch 75/100
2/2 [==============================] - 0s 230ms/step - loss: 15384748.6667 - val_loss: 2588429.2500
Epoch 76/100
2/2 [==============================] - 0s 241ms/step - loss: 15535781.6667 - val_loss: 2153216.0000
Epoch 77/100
2/2 [==============================] - 0s 230ms/step - loss: 15071853.3333 - val_loss: 2201847.7500
Epoch 78/100
2/2 [==============================] - 0s 252ms/step - loss: 14909302.6667 - val_loss: 2259406.2500
Epoch 79/100
2/2 [==============================] - 0s 249ms/step - loss: 14835822.6667 - val_loss: 2045863.5000
Epoch 80/100
2/2 [==============================] - 1s 293ms/step - loss: 14543187.3333 - val_loss: 2212336.2500
Epoch 81/100
2/2 [==============================] - 1s 283ms/step - loss: 14247691.3333 - val_lo

In [13]:
encoding = encoder.predict(inputs)
print(encoding.shape)

(36457, 64)


In [14]:
n_trn = train.shape[0]
df.reset_index(drop=True,inplace=True)

In [15]:
df_enc = pd.concat([df[feature_cols], pd.DataFrame(encoding, columns=[f'enc_{x}' for x in range(encoding_dim)])], axis=1)
df_enc['credit'] =train[TARGET]
train_enc = df_enc.iloc[:n_trn]
test_enc = df_enc.iloc[n_trn:]
print(train_enc.shape, test_enc.shape)

del train, test, df

(26457, 87) (10000, 87)


In [16]:
# train_enc = train_enc[train_enc['child_num']<=5].reset_index(drop=True) # outlier

In [17]:
train_enc.shape 

(26457, 87)

In [18]:
y = train_enc['credit']
train_enc.drop('credit',axis=1, inplace=True)
test_enc.drop('credit',axis=1, inplace=True)

# Modeling

In [19]:
def make_prediction(train, y, test, features, model, folds=5):

    kf = StratifiedKFold(n_splits=folds, random_state=random.randint(1,1000), shuffle=True)
    
    x_train = train[features]
    x_test = test[features]

    y_preds = np.zeros((x_test.shape[0],3))
    y_oof = np.zeros((x_train.shape[0],3))
    
    score = 0

    features_importance= pd.DataFrame({'Feature':[], 'Importance':[]})


    for fold, (tr_idx, val_idx) in enumerate(kf.split(x_train, y)):
        print(f'Fold: {fold+1}')

        x_tr, x_val = x_train.loc[tr_idx, features], x_train.loc[val_idx, features]
        y_tr, y_val = y[tr_idx], y[val_idx]

        print(x_tr.shape, x_val.shape)
        model.fit(x_tr, y_tr, eval_set=[(x_tr, y_tr),(x_val, y_val)], early_stopping_rounds = 50, verbose=50)
        
        fold_importance_df= pd.DataFrame({'Feature':[], 'Importance':[]})
        fold_importance_df['Feature']= features
        fold_importance_df['Importance']= model.feature_importances_
        fold_importance_df["fold"] = fold + 1
        features_importance = pd.concat([features_importance, fold_importance_df], axis=0)
        
        y_pred_val = model.predict_proba(x_val)
        y_oof[val_idx] = y_pred_val
        print(f"Fold {fold + 1} | log_loss Score: {log_loss(y_val, y_pred_val)}")

        score += log_loss(y_val, y_pred_val) / folds
        y_preds += model.predict_proba(x_test) / folds

        del x_tr, x_val, y_tr, y_val
        gc.collect()

    print(f"\nMean log_loss Score = {score}")
    print(f"OOF log_loss Score = {log_loss(y, y_oof)}")
    
#     return y_oof, y_preds, features_importance
    return y_preds

lgb_params = {
    'objective':'multiclass',
    'num_class': 3,
    'boosting_type':'gbdt',
    'metric':'multi_logloss',
    'n_jobs':-1,
    'learning_rate':0.01,
    'num_leaves': 1024,
    'max_depth':-1,
    'tree_learner':'feature',
    'colsample_bytree': 0.8,
    'subsample_freq':1,
    'subsample':0.8,
    'n_estimators':10000
}

xgb_params = {
    'n_estimators': 10000,
    'max_depth': 9,
    'learning_rate': 0.05,
    'booster': 'gbtree',
    'n_jobs': -1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'eval_metric':'mlogloss'
}

In [20]:
models = [
    LGBMClassifier(**lgb_params, seed=random.randint(1,10000)),
    LGBMClassifier(**lgb_params, seed=random.randint(1,10000)),
    XGBClassifier(**xgb_params, random_state=random.randint(1,10000)),
    XGBClassifier(**xgb_params, random_state=random.randint(1,10000))
]
# lgb_clf = LGBMClassifier(**lgb_params)
# xgb_clf = XGBClassifier(**xgb_params)
# y_oof_lgb, y_preds_lgb, fi_lgb = make_prediction(train_enc, y, test_enc, train_enc.columns, lgb_clf)
# y_oof_xgb, y_preds_xgb, fi_xgb = make_prediction(train_enc, y, test_enc, train_enc.columns, xgb_clf)

In [21]:
preds = []

for model in models:
    preds.append(make_prediction(train_enc, y, test_enc, train_enc.columns, model))

Fold: 1
(21165, 86) (5292, 86)
Training until validation scores don't improve for 50 rounds
[50]	training's multi_logloss: 0.699822	valid_1's multi_logloss: 0.787428
[100]	training's multi_logloss: 0.586156	valid_1's multi_logloss: 0.744082
[150]	training's multi_logloss: 0.503186	valid_1's multi_logloss: 0.721877
[200]	training's multi_logloss: 0.43995	valid_1's multi_logloss: 0.712655
[250]	training's multi_logloss: 0.389003	valid_1's multi_logloss: 0.710967
Early stopping, best iteration is:
[243]	training's multi_logloss: 0.395657	valid_1's multi_logloss: 0.710567
Fold 1 | log_loss Score: 0.7105669833783752
Fold: 2
(21165, 86) (5292, 86)
Training until validation scores don't improve for 50 rounds
[50]	training's multi_logloss: 0.699028	valid_1's multi_logloss: 0.792789
[100]	training's multi_logloss: 0.585048	valid_1's multi_logloss: 0.752675
[150]	training's multi_logloss: 0.501875	valid_1's multi_logloss: 0.733358
[200]	training's multi_logloss: 0.438132	valid_1's multi_logloss:

[200]	training's multi_logloss: 0.441075	valid_1's multi_logloss: 0.713455
[250]	training's multi_logloss: 0.389917	valid_1's multi_logloss: 0.710893
[300]	training's multi_logloss: 0.347246	valid_1's multi_logloss: 0.712222
Early stopping, best iteration is:
[270]	training's multi_logloss: 0.372058	valid_1's multi_logloss: 0.710771
Fold 3 | log_loss Score: 0.7107713898756125
Fold: 4
(21166, 86) (5291, 86)
Training until validation scores don't improve for 50 rounds
[50]	training's multi_logloss: 0.699068	valid_1's multi_logloss: 0.788275
[100]	training's multi_logloss: 0.584404	valid_1's multi_logloss: 0.745035
[150]	training's multi_logloss: 0.501652	valid_1's multi_logloss: 0.724291
[200]	training's multi_logloss: 0.437792	valid_1's multi_logloss: 0.716598
[250]	training's multi_logloss: 0.386474	valid_1's multi_logloss: 0.716293
Early stopping, best iteration is:
[216]	training's multi_logloss: 0.420354	valid_1's multi_logloss: 0.715641
Fold 4 | log_loss Score: 0.7156407817991176
F

[150]	validation_0-mlogloss:0.53135	validation_1-mlogloss:0.73961
[200]	validation_0-mlogloss:0.48623	validation_1-mlogloss:0.73160
[250]	validation_0-mlogloss:0.44548	validation_1-mlogloss:0.72767
[300]	validation_0-mlogloss:0.41131	validation_1-mlogloss:0.72629
[350]	validation_0-mlogloss:0.38130	validation_1-mlogloss:0.72638
[364]	validation_0-mlogloss:0.37470	validation_1-mlogloss:0.72692
Fold 5 | log_loss Score: 0.7258398347563987

Mean log_loss Score = 0.7136766005255091
OOF log_loss Score = 0.7136763773968463
Fold: 1
(21165, 86) (5292, 86)
[0]	validation_0-mlogloss:1.07623	validation_1-mlogloss:1.07880
[50]	validation_0-mlogloss:0.69442	validation_1-mlogloss:0.77685
[100]	validation_0-mlogloss:0.60448	validation_1-mlogloss:0.74268
[150]	validation_0-mlogloss:0.54580	validation_1-mlogloss:0.72545
[200]	validation_0-mlogloss:0.49983	validation_1-mlogloss:0.71462
[250]	validation_0-mlogloss:0.45791	validation_1-mlogloss:0.70768
[300]	validation_0-mlogloss:0.42398	validation_1-mlogl

In [22]:
import seaborn as sns
from matplotlib import pyplot as plt
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["Feature", "Importance"]].groupby("Feature").mean().sort_values(by="Importance", ascending=False)[:10].index
    best_features = feature_importance_df_[["Feature", "Importance"]].groupby("Feature").mean().sort_values(by="Importance", ascending=False)[:50]
    best_features.reset_index(inplace=True)
    print(best_features.dtypes)
    plt.figure(figsize=(8, 10))
    sns.barplot(x="Importance", y="Feature", data=best_features)
    plt.title('Features (avg over folds)')
    plt.tight_layout()
# display_importances(fi_lgb)

In [24]:
sum(preds)/len(preds)

array([[0.04929144, 0.16305611, 0.78765245],
       [0.21066283, 0.31866337, 0.4706738 ],
       [0.03900203, 0.05577508, 0.90522289],
       ...,
       [0.02724279, 0.08480119, 0.88795604],
       [0.12620324, 0.34721965, 0.52657711],
       [0.07840919, 0.14806037, 0.77353043]])

In [26]:
(sum(preds)/len(preds)).sum(axis=1)

array([1.        , 1.        , 1.        , ..., 1.00000002, 1.        ,
       0.99999999])

In [27]:
sub.iloc[:,1:] =  sum(preds)/len(preds)

In [28]:
sub.to_csv("submission/0513_ensemble_random.csv", index = False)